In [1]:
import os
from config import set_environment
set_environment()

In [2]:
import requests

url = "https://www.datalab.to/api/v1/marker"
filename = "F:/rag_sdk/datasets/alb_non_union_ben/files/pdf/ALB_NON_UNION_BEN.PDF"
output_dir = "./output/ALB_NON_UNION_BEN"
output_md = "./output/ALB_NON_UNION_BEN/ALB_NON_UNION_BEN.md"

In [3]:
form_data = {
    'file': ('test.pdf', open(filename, 'rb'), 'application/pdf'),
    "langs": (None, "en"),
    "force_ocr": (None, False)
}

headers = {"X-Api-Key": os.environ['DATALAB_API_KEY']}

response = requests.post(url, files=form_data, headers=headers)
data = response.json()

In [4]:
import time

max_polls = 100
check_url = data["request_check_url"]

for i in range(max_polls):
    time.sleep(60)
    response = requests.get(check_url, headers=headers)
    print(f"incomplete status - {response.status_code}")
    data = response.json()


    if data["status"] == "complete":
        break

incomplete status - 200


In [ ]:
data

In [5]:
import base64

# Function to fix base64 padding
def fix_base64_padding(b64_string):
    b64_string = b64_string.replace('\n', '')  # Remove any newline characters
    return b64_string + '=' * (-len(b64_string) % 4)

# Output the value of the 'markdown' element to a file named 'output.md'
with open(output_md, 'w') as markdown_file:
    markdown_file.write(data["markdown"])

# Output each element in the 'images' element to a separate file
for filename, content in data['images'].items():
    content_padded = fix_base64_padding(content)
    try:
        image_data = base64.b64decode(content_padded)
        with open(f'{output_dir}/{filename}', 'wb') as image_file:
            image_file.write(image_data)
    except Exception as e:
        print(f"Failed to decode {filename}: {e}")
